https://adventofcode.com/2022/day/19

Found an example of linear programming approach in https://www.reddit.com/r/adventofcode/comments/zpihwi/2022_day_19_solutions/


In [123]:
import re
from functools import reduce
import operator
import pulp as lp

In [2]:
with open("data/19.txt") as fh:
    data = fh.read()

In [5]:
testdata = """\
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian
"""

In [6]:
def load_data(data):
    L = []
    for line in data.strip().splitlines():
        bp, ore_ore, clay_ore, obs_ore, obs_clay, geode_ore, geode_obs = (
            int(i) for i in re.findall(r"[+-]?\d+", line)
        )
        D = {
            "bp": bp,
            "robots": {
                "ore": {"ore": ore_ore},
                "clay": {"ore": clay_ore},
                "obs": {"ore": obs_ore, "clay": obs_clay},
                "geode": {"ore": geode_ore, "obs": geode_obs},
            },
        }
        L.append(D)
    return L


load_data(testdata)

[{'bp': 1,
  'robots': {'ore': {'ore': 4},
   'clay': {'ore': 2},
   'obs': {'ore': 3, 'clay': 14},
   'geode': {'ore': 2, 'obs': 7}}},
 {'bp': 2,
  'robots': {'ore': {'ore': 2},
   'clay': {'ore': 3},
   'obs': {'ore': 3, 'clay': 8},
   'geode': {'ore': 3, 'obs': 12}}}]

In [7]:
bps = load_data(testdata)
bps

[{'bp': 1,
  'robots': {'ore': {'ore': 4},
   'clay': {'ore': 2},
   'obs': {'ore': 3, 'clay': 14},
   'geode': {'ore': 2, 'obs': 7}}},
 {'bp': 2,
  'robots': {'ore': {'ore': 2},
   'clay': {'ore': 3},
   'obs': {'ore': 3, 'clay': 8},
   'geode': {'ore': 3, 'obs': 12}}}]

In [8]:
bps[0]["robots"]

{'ore': {'ore': 4},
 'clay': {'ore': 2},
 'obs': {'ore': 3, 'clay': 14},
 'geode': {'ore': 2, 'obs': 7}}

In [116]:
def collect_geodes(bp, timelimit=24):
    problem = lp.LpProblem(f"problem:{bp['bp']:02d}", lp.LpMaximize)
    robots_geo = [
        lp.LpVariable(f"robots_geo{i:02d}", 0, 0 if i == 0 else None, lp.LpInteger)
        for i in range(timelimit)
    ]
    problem += sum(robots_geo)

    robots_ore = []
    robots_cly = []
    robots_obs = []
    onhand_ore = []
    onhand_cly = []
    onhand_obs = []
    build_ore = []
    build_cly = []
    build_obs = []
    build_geo = []

    for i in range(timelimit):
        robots_ore.append(
            lp.LpVariable(f"robots_ore{i:02d}", 1 if i == 0 else 0, 1 if i == 0 else None, lp.LpInteger)
        )
        robots_cly.append(
            lp.LpVariable(f"robots_cly{i:02d}", 0, 0 if i == 0 else None, lp.LpInteger)
        )
        robots_obs.append(
            lp.LpVariable(f"robots_obs{i:02d}", 0, 0 if i == 0 else None, lp.LpInteger)
        )
        onhand_ore.append(
            lp.LpVariable(f"onhand_ore{i:02d}", 0, 0 if i == 0 else None, lp.LpInteger)
        )
        onhand_cly.append(
            lp.LpVariable(f"onhand_cly{i:02d}", 0, 0 if i == 0 else None, lp.LpInteger)
        )
        onhand_obs.append(
            lp.LpVariable(f"onhand_obs{i:02d}", 0, 0 if i == 0 else None, lp.LpInteger)
        )
        build_ore.append(lp.LpVariable(f"build_ore{i:02d}", 0, 1, lp.LpInteger))
        build_cly.append(lp.LpVariable(f"build_cly{i:02d}", 0, 1, lp.LpInteger))
        build_obs.append(lp.LpVariable(f"build_obs{i:02d}", 0, 1, lp.LpInteger))
        build_geo.append(lp.LpVariable(f"build_geo{i:02d}", 0, 1, lp.LpInteger))

        # Build zero or one robot per turn
        problem += build_ore[-1] + build_cly[-1] + build_obs[-1] + build_geo[-1] <= 1

        if i != 0:
            # Robot costs
            problem += onhand_ore[-2] - build_ore[-1] * bp["robots"]["ore"]["ore"] >= 0
            problem += onhand_ore[-2] - build_cly[-1] * bp["robots"]["clay"]["ore"] >= 0
            problem += onhand_ore[-2] - build_obs[-1] * bp["robots"]["obs"]["ore"] >= 0
            problem += onhand_cly[-2] - build_obs[-1] * bp["robots"]["obs"]["clay"] >= 0
            problem += (
                onhand_ore[-2] - build_geo[-1] * bp["robots"]["geode"]["ore"] >= 0
            )
            problem += (
                onhand_obs[-2] - build_geo[-1] * bp["robots"]["geode"]["obs"] >= 0
            )
            # Resources on hand
            problem += (
                onhand_ore[-1]
                == onhand_ore[-2]
                + robots_ore[-2]
                - build_ore[-1] * bp["robots"]["ore"]["ore"]
                - build_cly[-1] * bp["robots"]["clay"]["ore"]
                - build_obs[-1] * bp["robots"]["obs"]["ore"]
                - build_geo[-1] * bp["robots"]["geode"]["ore"]
            )
            problem += (
                onhand_cly[-1]
                == onhand_cly[-2]
                + robots_cly[-2]
                - build_obs[-1] * bp["robots"]["obs"]["clay"]
            )
            problem += (
                onhand_obs[-1]
                == onhand_obs[-2]
                + robots_obs[-2]
                - build_geo[-1] * bp["robots"]["geode"]["obs"]
            )
            # Number of robots
            problem += robots_ore[-1] == robots_ore[-2] + build_ore[-1]
            problem += robots_cly[-1] == robots_cly[-2] + build_cly[-1]
            problem += robots_obs[-1] == robots_obs[-2] + build_obs[-1]
            problem += robots_geo[i] == robots_geo[i-1] + build_geo[-1]

    status = problem.solve()
    return problem.objective.value()


In [121]:
%%time
bps = load_data(data)
sum(bp['bp'] * collect_geodes(bp, 24) for bp in bps)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/wleftwich/.pyenv/versions/3.11.0/envs/aoc/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e56c66ba2ca04e6b9ea16fdb222218da-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/e56c66ba2ca04e6b9ea16fdb222218da-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 328 COLUMNS
At line 1874 RHS
At line 2198 BOUNDS
At line 2463 ENDATA
Problem MODEL has 323 rows, 264 columns and 993 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5.80994 - 0.00 seconds
Cgl0003I 4 fixed, 95 tightened bounds, 25 strengthened rows, 0 substitutions
Cgl0003I 2 fixed, 52 tightened bounds, 45 strengthened rows, 0 substitutions
Cgl0003I 1 fixed, 14 tightened bounds, 38 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 9 tightened bounds, 28 strengthened rows, 0 substitutions
Cgl0003

1389.0

In [125]:
%%time
reduce(operator.mul, (collect_geodes(bp, 32) for bp in bps[:3]))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/wleftwich/.pyenv/versions/3.11.0/envs/aoc/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/7868cb7420f5449cb849d150a127ea33-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/7868cb7420f5449cb849d150a127ea33-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 440 COLUMNS
At line 2514 RHS
At line 2950 BOUNDS
At line 3303 ENDATA
Problem MODEL has 435 rows, 352 columns and 1337 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 26.3359 - 0.00 seconds
Cgl0003I 3 fixed, 149 tightened bounds, 33 strengthened rows, 0 substitutions
Cgl0003I 3 fixed, 89 tightened bounds, 31 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 44 tightened bounds, 53 strengthened rows, 0 substitutions
Cgl0003I 1 fixed, 31 tightened bounds, 44 strengthened rows, 0 substitutions
Cgl0

3003.0